In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import sklearn
import sys
import time
from tensorflow import keras
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    print(dirname)
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.
print(tf.__version__)

2.1.0


In [5]:
train_dir = "10monkey/training"
train_generator = keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=keras.applications.resnet50.preprocess_input,
    zoom_range=0.2,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest'
)
train_gen = train_generator.flow_from_directory(directory=train_dir,
                                                target_size=(224, 224), 
                                                class_mode='categorical',
                                                batch_size=32, 
                                                shuffle=True, 
                                                seed=7, 
                                                interpolation='nearest')

Found 1098 images belonging to 10 classes.


In [6]:
validation_dir = "10monkey/validation"
validation_gen = keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=keras.applications.resnet50.preprocess_input,
)
validation_generator = validation_gen.flow_from_directory(directory=validation_dir,
                                                target_size=(224, 224), 
                                                class_mode='categorical',
                                                batch_size=32, 
                                                seed = 7,
                                                shuffle=False,
                                               )

Found 272 images belonging to 10 classes.


In [ ]:
model = keras.models.Sequential()
model.add(
    keras.applications.resnet50.ResNet50(include_top=False,
                                weights='imagenet',
                                pooling='avg',)
)
model.add(
    keras.layers.Dense(10, activation='softmax')
)
model.layers[0].trainable=False
model.compile(loss="categorical_crossentropy", optimizer='sgd',
             metrics=['accuracy'])

15622144/94765736 [===>..........................] - ETA: 9:02:08

In [ ]:
epochs=20
history = model.fit_generator(train_gen,
                               steps_per_epoch=train_generator.samples // 32,
                                epochs=epochs,
                               validation_data=validation_generator,
                               validation_steps=validation_generator.samples // 32
                               )